In [11]:
from flask import Flask, request, flash, redirect, url_for
import json
import cv2
import numpy as np
from urllib.request import urlopen

In [ ]:
ace_cascade = cv2.CascadeClassifier('haar/haarcascade_frontalface.xml')


def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in {'png', 'jpg', 'jpeg', 'gif'}

def convert(o):
    if isinstance(o, int32): return int(o)  
    raise TypeError

def DetectionOpenCV(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    allFaces = []

    for (x, y, w, h) in faces:
        data = {
            "top": int(x),
            "left": int(y),
            "width": int(w),
            "heignt" : int(h)
        }
        
        allFaces.append(data)
        print(allFaces)
    return allFaces


def LoadImageFromUrl(url):
    resp = urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    img = cv2.imdecode(image, cv2.IMREAD_COLOR)
    # Load the cascade
    return DetectionOpenCV(img)

def LoadImageFromFile():
    inputName = 'famtest.jpg'
    img = cv2.imread(inputName)

    return DetectionOpenCV(img)

app = Flask(__name__)

@app.route("/")
def hello():
  return "Hello World!"

@app.route("/detectFacesFromFile")
def detectFacesFile():
    return json.dumps(LoadImageFromFile(), default=convert)


@app.route("/detectFacesFromUrl")
def detectFacesUrl():
    return json.dumps(LoadImageFromUrl(request.args.get("url")), default=convert)

@app.route('/detectFacesFromUpload', methods=['GET', 'POST'])
def detectFacesUpload():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file and allowed_file(file.filename):
            img = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_UNCHANGED)
            return json.dumps(DetectionOpenCV(img), default=convert)
    return '''
        <!doctype html>
        <title>Upload new File</title>
        <h1>Upload new File</h1>
        <form method=post enctype=multipart/form-data>
        <input type=file name=file>
        <input type=submit value=Upload>
        </form>
    '''

if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Nov/2020 14:06:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2020 14:07:05] "GET /detectFacesFromUpload HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2020 14:07:34] "POST /detectFacesFromUpload HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2020 14:07:35] "GET /detectFacesFromUpload HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2020 14:07:41] "POST /detectFacesFromUpload HTTP/1.1" 200 -


[{'top': 197, 'left': 121, 'width': 107, 'heignt': 107}]
[{'top': 197, 'left': 121, 'width': 107, 'heignt': 107}, {'top': 539, 'left': 167, 'width': 73, 'heignt': 73}]


127.0.0.1 - - [18/Nov/2020 14:09:56] "GET /detectFacesFromUrl?url=http://lorempixel.com/output/people-q-c-500-500-2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2020 14:10:28] "GET //detectFacesFromUpload HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2020 14:10:33] "GET /.detectFacesFromUpload HTTP/1.1" 404 -
127.0.0.1 - - [18/Nov/2020 14:10:36] "GET //detectFacesFromUrl?url=http://lorempixel.com/output/people-q-c-500-500-2.jpg HTTP/1.1" 200 -


In [3]:
import cv2

face_cascade = cv2.CascadeClassifier('faces.xml')
inputName = 'famtest.jpg'

img = cv2.imread(inputName)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.1, 4)
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    cv2.putText(img, str(x) + "," + str(y)+ "," + str(x+w)+ "," + str(y+h),(x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

cv2.imshow('img', img)
cv2.waitKey()

-1